# **Creación de un corpus textual de poesía**

## Descripción del corpus

Este trabajo consste en la creación de un corpus de poesía, basada en la extracción automática de poemas contenidos en el sitio web Poemas del Alma (https://www.poemas-del-alma.com/).

Esta fuente contiene un amplio repositorio de poesía en español, organizada alfabéticamente por autor.

Cada documento del corpus corresponde a un poema individual, incluyendo el contenido textual asi como los metadatos: Autor, título y url.

## Extracción de los datos

Se realizón la obtención de datos mediante web scraping.

**Visión General del Proceso**

El sistema extrae información de poemas mediante un proceso en cascada de cuatro niveles:



1.   Nivel Alfabético → Páginas índice por letra (a-z).
2.   Nivel Poeta → Listados de poetas individuales.
3.   Nivel Poema → Listados de poemas de cada poeta.
4.   Nivel texto → Contenido completo de cada poema.


## Nivel Alfabético
En esta etapa se generan los URL´s para acceder al abececdario de autores que contiene la página.

In [ ]:
# Librerias necesarias
import requests
from bs4 import BeautifulSoup
import pandas as pd
import string
from urllib.parse import urljoin

In [ ]:
# Datos de la página
url = "https://www.poemas-del-alma.com/"

headers = {
    "User-Agent": "Mozilla/5.0"
}

In [ ]:
def urls_alfa(url_base: str, extension: str = ".html") -> list[str]:
    """
    Extrae todos los URLs contenidos en el alfabeto de la página.

    Args:
        urls_base: URL base de la página
        extensión: Extensión de los archivos
    Returns:
        Lista con urls de cada laetra el abecedario
    """
    return [f"{url_base}{letra}{extension}" for letra in string.ascii_lowercase]

**Nota:** Es importante hacer una revisión de la página en HTML para saber qué extraer.




## Nivel Poeta.
Basado en la lista de URLs generada por el nivel alfabético se extraen los URLs
correspondientes a cada autor o poeta en orden alfabético.

**Proceso de extracción**

1.   Navega cada URL alfabética.
2.   Localiza elemento con `id="content"`.
3.   Extrae todos los enlaces `<a>`.
4.   Filtra enlaces válidos.
5.   Normaliza URLs que inician con `//` agregando `https:`

In [ ]:
def obtener_poetas(urls: list[str])  -> pd.DataFrame:
    """
    Extrae información de poetas desde una lista de URLs.

    Args:
        urls: Lista de URLs con listados alfabéticos de poetas

    Returns:
        DataFrame con columnas 'nombre' y 'url' de cada poeta
    """
    data = []

    for url in urls:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        links = soup.find(id="content").find_all("a")

        for a in links:
            texto = a.get_text(strip=True)

            if not a.find("strong") and texto and len(texto) > 1:
                href = a.get("href")
                if href and href.startswith("//"):
                    href = "https:" + href

                data.append({
                    "nombre": texto,
                    "url": href
                })
    return pd.DataFrame(data)

## Nivel poema
Una vez tenemos un dataframe de nombre y URLs de los autores, se obtienen los URLs correspondientes a cada poema de cada poeta.

**Proceso de extracción**



1.   Itera sobre cada poeta del DataFrame anterior.
2.   Accede a la URL del poeta.
3.   Localiza elemento con `id="content"`.
4.   Extrae enlaces a poemas individuales.
5.   Filtra por extensión `.htm` y URLs relativas (no `//`).
6.   Construye URLs usando `urljoin()`.


In [ ]:
def obtener_poemas(df_poetas: pd.DataFrame) -> pd.DataFrame:
    """
    Extrae información de poemas desde una DF que contiene
     links de los poetas.

    Args:
        df_poetas: DF de URLs de poetas

    Returns:
        DataFrame con columnas 'nombre', 'url_poeta' y 'url_poema' de cada poema
    """
    data = []

    for _, fila in df_poetas.iterrows():
        response = requests.get(fila["url"], headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        content = soup.find(id="content").find_all("a")

        for a in content:
            href = a.get("href")

            if not href:
                continue

            if href.endswith(".htm") and not href.startswith("//"):

              data.append({
                  "nombre": fila["nombre"],
                  "url_poeta": fila["url"],
                  "poemas": urljoin(response.url, href)
              })

    return pd.DataFrame(data)

## Nivel texto
Finalmente, con el dataframe de URLs de los poemas podemos realizar la identificación y extracción de cada documento.


1.   Itera sobre cada poema de dataframe recibido.
2.   Accede al URL de cada poema.
3.   Extrae el título de cda poema (`h1`).
4.   Identifica el elemento `id = "content"`.
5.   Se procesa el HTML y preservando formaro:
>*   `<br><br>` Doble salto de línea `\n\n`.
>*   `<br /> y <br>` Salto de línea simple `\n`.
6.   Convierte HTML en texto plano.

In [ ]:
def obtener_texto(df_poemas: pd.DataFrame) -> pd.DataFrame:
    """
    Extrae el texto desde un DF que contiene links de
    poemas de cada autor.

    Args:
        df_poemas: DF de URLs de poemas.

    Returns:
        DataFrame con columnas 'autor, 'titulo' y 'poema'de cada poema por cada autor.
    """
  data = []
  for _, fila in df_poemas.iterrows():
    response = requests.get(fila["poemas"], headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    titulo = soup.find("h1")
    content= soup.find(id="content")

    if not titulo or not content:
      continue

    html_poema = content.decode_contents()
    html_poema = html_poema.replace("<br><br>", "\n\n")
    html_poema = html_poema.replace("<br />", "\n")
    html_poema = html_poema.replace("<br>", "\n")
    poema = BeautifulSoup(html_poema, "html.parser").get_text()

    data.append({
        "autor": fila["nombre"],
        "titulo": titulo.get_text(strip=True),
        "poemas": poema.strip()
    })
  return pd.DataFrame(data)

## Resultados
El flujo de datos de la extracción es el siguiente:


```
    URLs Alfabéticas
             ↓
  DataFrame Poetas (n registros)
       [nombre, url]
             ↓
  DataFrame Poemas (m registros)
      [nombre, url_poeta, poemas]
             ↓
  DataFrame Final (m registros)
    [autor, titulo, poemas]
```



In [ ]:
# URLs alfabéticas
urls = urls_alfa(url)
# URLs de poetas
df_poetas = obtener_poetas(urls)
# URLs de poemas
df_poemas = obtener_poemas(df_poetas)
# Extracción de documentos
total_poemas = obtener_texto(df_poemas)

**Visualización de la tabla final**

In [ ]:
total_poemas.head(5)

,autor,titulo,poemas
0,A Isa Ahmad Ibn Muhammad Ibn Qadim,Panegírico a Al-Muzaffar,Poema siguiente\n\r\n¡Que Dios te muestre lo q...
1,A Isa Ahmad Ibn Muhammad Ibn Qadim,Poema que escribió a un poeta que no le agradaba,"Poema siguiente\n\r\nLeona soy, pero no me agr..."
2,Abel Alarcón,La abadesa,Poema siguiente\n\r\nPor el jardín paseaba la ...
3,Abel Alarcón,Pascua,"Poema siguiente\n\r\nElevó, adusto, el sacerdo..."
4,Abel G. Fagundo,Extraña salvaje,Poema siguiente\n\r\nYo tengo una extraña que ...


In [ ]:
print('Extracción total: ',total_poemas.shape)

Extracción total:  (13518, 3)


In [ ]:
# Ejemplo de documento
print(total_poemas['poemas'][0])

'Poema siguiente\n\r\n¡Que Dios te muestre lo que deseas\r\ny que no cesen de aumentar tus altos hechos!\r\nLos signos de su rostro y su horóscopo\r\nmuestran lo que puedes esperar de él;\r\nlos corceles le ansían, las espadas le anhelan\r\ny relucen por él los estandartes;\r\nte parecerá luna en lo mas alto del cielo\r\ny sus estrellas son los ejércitos.\r\n¿Acaso podría ser de otra manera un cachorro\r\nque engendraron para la gloria leones sementales?\r\nSois los Amiríes, el más noble linaje;\r\nvuestros vástagos y vuestros antepasados, intachables;\r\nvuestros jóvenens son prudentes como ancianos;\r\nvuestros ancianos valientes como jóvenes.\n\n ME GUSTA 2 \n\n\n\nVer métrica Poema siguiente\xa0\n\xa0Volver a A Isa Ahmad Ibn Muhammad Ibn Qadim'

## Conclusión
Se logró la extracción automática de documentos (poemas) conservando características relevantes como son el salto y el doble salto de línea.

Se recopilaron un total de 13 518 poemas.


# Consideraciones
Las limitaciones en esta extracción fueron la estructura interna de la página, ya que puede haber más poemas dentro de los URLs de los poemas en el cual se tendría que realizar un nivel más de extracción.

El contenido de este corpus es llevado a cabo respetando el acceso público a los textos y utilizando el corpus exclusivamente con fines académicos y de investigación.

In [ ]:
#obtener_texto(df_poemas).to_csv("poemas_completo.csv", index=False)